In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time

L'objectif de ce TP est de créer un modèle NN simple (MLP), qui utilise des couches "fully connected", pour faire la classification de chiffres manuscrites du dataset MNIST.

# Dataset

Importer et préparer le dataset MNIST

In [3]:
# load the training dataset.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# reshape the data by flattening the images
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples from the training data for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

11501568/11490434 [==============================] - 2s 0us/step


# Modèle

Nous voulons construire un modèle MLP (multi layer perceptron).
Pour cela, nous avons besoin d'abord de définir les couches de perceptrons qui vont composer le modèle, c'est à dire de couches "fully connected" qui prendent en entrée un vecteur de valeurs $x$ et qui donnent en sortie une activation $a = f(W*x + b)$, où:

* $W$ est la matrice des poids
* $b$ est le vecteur de biais
* $f(.)$ est la fonction d'activation

Nous allons créer une classe "FCLayer" pour des objets de type "fully connected layer".
La création se fait par "subclassing" de la classe générique "tf.keras.layers.Layer" utilisée pour créer des couches customisées dans Keras

In [4]:
# création d'une classe FCLayer pour des objets de type "fully connected layer"

class FCLayer(tf.keras.layers.Layer):
    
    # fonction pour initiliser un objet de la classe
    def __init__(self, units=32, input_dim=32, use_relu=True):
        super(FCLayer, self).__init__()
        
        # définition de la variable de la classe
        self.use_relu = use_relu
        
        # definition des poids W
        w_init = tf.random_normal_initializer()  # initialisation aléatoire normale
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        
        # definition du biais b
        b_init = tf.zeros_initializer()  # initialisation à zéro
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    # pour calculer la sortie du layer
    def call(self, inputs):
        
        # calcul des sorties
        layer = tf.matmul(inputs, self.w) + self.b
        
        # utiliser activation si c'est le cas
        if self.use_relu:
            layer = tf.nn.relu(layer)
        return layer

Maintenant que nous avons à disposition les briques pour construire notre modèle, nous pouvons voir comment faire cela dans Keras.
Nous allons construire un modèle qui prend en entrée les images aplaties (flatten), c'est à dire des vecteurs de taille 784, ensuite il est composé de deux couches "fully connected", chacune de 32 neurones avec activation ReLU, et une couche finale de classification composée par un nombre de neurones égal au nombre de classes, et sans activation, pour pouvoir donner en sortie la valeur de logit pour chaque classe.

In [5]:
# create a NN model with the FCLayer
input_shape = 784

inputs = keras.Input(shape=(input_shape,), name="digits")
x1 = FCLayer(units=32, input_dim=input_shape, use_relu=True)(inputs)
x2 = FCLayer(units=32, input_dim=32, use_relu=True)(x1)
outputs = FCLayer(units=10, input_dim=32, use_relu=False)(x2)

model = keras.Model(inputs=inputs, outputs=outputs)

2021-11-13 16:31:34.548179: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-13 16:31:34.548301: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-13 16:31:34.548365: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-DMWORK): /proc/driver/nvidia/version does not exist
2021-11-13 16:31:34.549273: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Entrainement

Entraînons le modèle en utilisant une méthode déscente de gradients stochastique (avec moni-batches) avec une boucle d'entraînement personnalisée.

Tout d'abord, nous aurons besoin d'un optimiseur et d'une loss function. Nous choisissons un optimiseur SGD et une loss Categorical cross entropy, comme il s'agit d'un problème de classification

In [6]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

En ce qui concerne les métriques d'évaluation du modèle, comme il s'agit d'un problème de classification nous utilison une métrique "categorical cross-entropy"

In [7]:
# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

Préparation du dataset : division en mini-batches pour pouvoir appliquer un algorithme de déscente de gradient stochastique. Pour cela, nous utilisons la fonction tf.data.Dataset.from_tensor_slices, une fonction TF2 qui gère automatiquement les données et leur division en minibatches

In [8]:
batch_size = 64


# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

2021-11-13 16:31:35.880770: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39200000 exceeds 10% of free system memory.


Voici notre boucle d'entraînement :

* Nous ouvrons une boucle qui parcourt les épochs
* Pour chaque époque, nous ouvrons une boucle qui itère sur l'ensemble de données, en mini-batches
* Pour chaque mini-batch, nous ouvrons un GradientTape(), une fonctionnalité de tensorflow 2.X pour pouvoir faire la backpropagation et le calcul des gradients.
* A l'intérieur du GradientTape, nous appelons le modèle (forward pass) et calculons la loss function
* En dehors du GradientTape, on récupère les gradients des poids du modèle par rapport à la loss
* Enfin, nous utilisons l'optimiseur pour mettre à jour les poids du modèle en fonction des gradients
* Ensuite, pour chaque epoch nous allons calculer et visualiser des métriques pour monitorer l'état d'anacement de l'entrainement et la qualité du modèle

In [9]:
epochs = 20

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0


2021-11-13 16:31:39.498628: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39200000 exceeds 10% of free system memory.


Training loss (for one batch) at step 0: 6.4795
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.0686
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.8814
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.3087
Seen so far: 38464 samples
Training acc over epoch: 0.7339
Validation acc: 0.8445
Time taken: 20.31s

Start of epoch 1
Training loss (for one batch) at step 0: 0.3739
Seen so far: 64 samples


2021-11-13 16:31:59.808510: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39200000 exceeds 10% of free system memory.


Training loss (for one batch) at step 200: 0.4945
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.5349
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.3663
Seen so far: 38464 samples
Training acc over epoch: 0.8664
Validation acc: 0.8786
Time taken: 16.87s

Start of epoch 2
Training loss (for one batch) at step 0: 0.5524
Seen so far: 64 samples


2021-11-13 16:32:16.684864: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39200000 exceeds 10% of free system memory.


Training loss (for one batch) at step 200: 0.4085
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.2477
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.2795
Seen so far: 38464 samples
Training acc over epoch: 0.8930
Validation acc: 0.9033
Time taken: 19.56s

Start of epoch 3
Training loss (for one batch) at step 0: 0.3155
Seen so far: 64 samples


2021-11-13 16:32:36.236535: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 39200000 exceeds 10% of free system memory.


Training loss (for one batch) at step 200: 0.3672
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.1930
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.2005
Seen so far: 38464 samples
Training acc over epoch: 0.9060
Validation acc: 0.9092
Time taken: 28.34s

Start of epoch 4
Training loss (for one batch) at step 0: 0.7392
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.3624
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.2799
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.2028
Seen so far: 38464 samples
Training acc over epoch: 0.9163
Validation acc: 0.9250
Time taken: 39.05s

Start of epoch 5
Training loss (for one batch) at step 0: 0.1357
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.2995
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.1909
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.2732
Seen s

## Biblio

* https://www.tensorflow.org/guide/keras/custom_layers_and_models

* https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch